# Projeto de Fine-tuning YOLOv5 para Semáforos e Placas de Pare

Este projeto demonstra o fine-tuning de um modelo YOLOv5s pré-treinado para detectar especificamente "semáforos" e "placas de pare" usando um subset do dataset COCO.

## Objetivo

O objetivo principal foi adaptar um modelo de detecção de objetos YOLOv5 para reconhecer apenas duas classes específicas (`traffic light` e `stop sign`) a partir do dataset COCO128, superando os desafios encontrados durante o processo de configuração do ambiente e dos dados.

## Dataset

* **Nome:** COCO128 (subset do dataset COCO)
* **Classes alvo para treinamento:**
    * `traffic light` (ID original COCO: 9, remapeado para: 0)
    * `stop sign` (ID original COCO: 13, remapeado para: 1)
* **Preparação:** Devido às incompatibilidades de IDs de classes entre o dataset COCO128 original e a configuração de `nc=2` do modelo, um script Python (`prepare_custom_dataset.py`) foi utilizado para filtrar e remapear os rótulos, criando uma estrutura de dataset customizada (`traffic_signs_coco128`) que o YOLOv5 pode processar corretamente.

## Ambiente de Desenvolvimento

* **Plataforma:** Google Colab (GPU: NVIDIA A100-SXM4-40GB)
* **Framework:** PyTorch
* **Modelo Base:** YOLOv5s (pesos pré-treinados em `yolov5s.pt`)
* **Principais Desafios e Soluções:**
    * **Prompt do Weights & Biases (W&B):** Inicialmente, o `wandb` persistia solicitando autenticação. A solução mais eficaz foi desinstalá-lo via `!pip uninstall -y wandb` na pipeline de configuração, embora o prompt ainda aparecesse e fosse contornado com a seleção manual da opção "Don't visualize my results".
    * **Argumentos não reconhecidos:** O argumento `--disable-wandb` causou erro de `unrecognized arguments`, sendo removido do comando de treinamento.
    * **Erro de IDs de Classe:** O dataset COCO128 continha rótulos com IDs de classes que excediam o `nc=2` configurado (`Label class 79 exceeds nc=2`). Isso foi resolvido com o script `prepare_custom_dataset.py`, que filtra e remapeia as classes 9 e 13 para 0 e 1, respectivamente.
    * **Imagens de Validação Ausentes:** O dataset `coco128` não possui uma pasta `val2017` separada. O arquivo `data/coco_traffic_signs.yaml` foi configurado para usar as imagens de `train2017` tanto para treinamento quanto para validação (`val: images/train2017`), permitindo que o YOLOv5 fizesse a divisão interna.

## Treinamento

* **Épocas:** 100
* **Tamanho da Imagem:** 640x640 pixels
* **Batch Size:** 16
* **Comando de Treinamento:**
    ```bash
    python train.py --img 640 --batch 16 --epochs 100 --data 'data/coco_traffic_signs.yaml' --weights yolov5s.pt --name traffic_signs_run --project runs/train
    ```

## Resultados do Treinamento

O treinamento foi concluído com sucesso. Abaixo estão as métricas de validação para as 100 épocas:

| Classe          | Precision (P) | Recall (R) | mAP50 | mAP50-95 | Instâncias |
| :-------------- | :------------ | :--------- | :---- | :------- | :--------- |
| **Geral (all)** | 0.00691       | 0.857      | 0.239 | 0.171    | 23         |
| traffic light   | 0.0105        | 0.714      | 0.176 | 0.141    | 14         |
| stop sign       | 0.00328       | 1.000      | 0.302 | 0.201    | 9          |

* **Observações:** As métricas de mAP são baixas, o que é esperado para um modelo treinado em um dataset muito pequeno (128 imagens no total, com poucas instâncias das classes alvo). O Recall de 1.0 para 'stop sign' indica que todas as placas de pare foram detectadas, embora a precisão tenha sido muito baixa.
* **Pesos do Modelo:** Os modelos treinados (`last.pt` e `best.pt`) estão salvos no diretório `runs/train/traffic_signs_run/weights/`.

## Próximos Passos (Sugestões para futuras melhorias)

* Treinar com um dataset maior e mais diversificado de semáforos e placas de pare.
* Realizar mais épocas de treinamento.
* Experimentar técnicas de aumento de dados (data augmentation) ou ajustes de hiperparâmetros.
* Avaliar o modelo em imagens ou vídeos reais.

In [1]:
# 1. Clonar o repositório YOLOv5
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 17516 (delta 6), reused 0 (delta 0), pack-reused 17497 (from 4)
Receiving objects: 100% (17516/17516), 16.61 MiB | 30.04 MiB/s, done.
Resolving deltas: 100% (11994/11994), done.


In [2]:
# 2. Entrar na pasta do YOLOv5
%cd yolov5

# 3. Instalar as dependências necessárias
!pip install -r requirements.txt

/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82


In [3]:
# 4. Desinstalar Weights & Biases para evitar prompts
!pip uninstall -y wandb

Found existing installation: wandb 0.21.0
Uninstalling wandb-0.21.0:
  Successfully uninstalled wandb-0.21.0


In [4]:
# 5. Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 6. Baixar e descompactar o dataset coco128
# Será descompactado em /content/datasets/coco128
!wget https://ultralytics.com/assets/coco128.zip -P /content/datasets/
!unzip -q /content/datasets/coco128.zip -d /content/datasets/

--2025-07-18 14:31:30--  https://ultralytics.com/assets/coco128.zip
Resolving ultralytics.com (ultralytics.com)... 99.83.190.102, 75.2.70.75
Connecting to ultralytics.com (ultralytics.com)|99.83.190.102|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.ultralytics.com/assets/coco128.zip [following]
--2025-07-18 14:31:31--  https://www.ultralytics.com/assets/coco128.zip
Resolving www.ultralytics.com (www.ultralytics.com)... 172.64.147.154, 104.18.40.102, 2606:4700:4400::ac40:939a, ...
Connecting to www.ultralytics.com (www.ultralytics.com)|172.64.147.154|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/ultralytics/assets/releases/download/v0.0.0/coco128.zip [following]
--2025-07-18 14:31:31--  https://github.com/ultralytics/assets/releases/download/v0.0.0/coco128.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... 

In [6]:
# 7. Criar script Python para preparar o dataset customizado
%%writefile prepare_custom_dataset.py
import os
import shutil
from pathlib import Path

# Define original COCO IDs and their new desired IDs
COCO_ID_MAP = {
    9: 0,  # traffic light -> new class 0
    13: 1, # stop sign -> new class 1
}

# Base paths (relative to /content/)
original_coco128_path = Path('/content/datasets/coco128')
new_custom_dataset_path = Path('/content/datasets/traffic_signs_coco128')

print(f"Preparando dataset customizado para {COCO_ID_MAP} de {original_coco128_path} para {new_custom_dataset_path}...")

# 1. Criar a nova estrutura de dataset
new_custom_dataset_path.mkdir(parents=True, exist_ok=True)
(new_custom_dataset_path / 'images').mkdir(exist_ok=True)
(new_custom_dataset_path / 'labels').mkdir(exist_ok=True)

# 2. Copiar imagens para a nova estrutura de dataset
print("Copiando imagens...")
for split_dir in ['train2017', 'val2017']:
    # Garantir que os diretórios de destino existem
    (new_custom_dataset_path / 'images' / split_dir).mkdir(parents=True, exist_ok=True)
    (new_custom_dataset_path / 'labels' / split_dir).mkdir(parents=True, exist_ok=True) # Labels serão escritos aqui

    original_images_split_path = original_coco128_path / 'images' / split_dir
    if original_images_split_path.exists():
        for img_file in original_images_split_path.glob('*.jpg'):
            shutil.copy(img_file, new_custom_dataset_path / 'images' / split_dir / img_file.name)
    else:
        print(f"Aviso: {original_images_split_path} não encontrado. Certifique-se de que coco128 foi descompactado corretamente.")

# 3. Filtrar e remapear rótulos
print("Filtrando e remapeando rótulos...")
for split_dir in ['train2017', 'val2017']:
    current_original_labels_dir = original_coco128_path / 'labels' / split_dir
    current_output_labels_dir = new_custom_dataset_path / 'labels' / split_dir

    if current_original_labels_dir.exists():
        for label_file in current_original_labels_dir.glob('*.txt'):
            filtered_lines = []
            with open(label_file, 'r') as f_in:
                for line in f_in:
                    parts = line.strip().split()
                    if not parts:
                        continue
                    original_class_id = int(parts[0])

                    if original_class_id in COCO_ID_MAP:
                        new_class_id = COCO_ID_MAP[original_class_id]
                        # Reescrever a linha com o novo ID da classe
                        filtered_lines.append(f"{new_class_id} {' '.join(parts[1:])}\n")

            if filtered_lines:
                with open(current_output_labels_dir / label_file.name, 'w') as f_out:
                    f_out.writelines(filtered_lines)
            else:
                # Se nenhum rótulo relevante for encontrado, criar um arquivo vazio
                (current_output_labels_dir / label_file.name).touch()
    else:
        print(f"Aviso: {current_original_labels_dir} não encontrado. Certifique-se de que os rótulos de coco128 existem.")

print("Preparação do dataset customizado concluída.")

Writing prepare_custom_dataset.py


In [7]:
# 8. Executar o script de preparação do dataset
!python prepare_custom_dataset.py

Preparando dataset customizado para {9: 0, 13: 1} de /content/datasets/coco128 para /content/datasets/traffic_signs_coco128...
Copiando imagens...
Aviso: /content/datasets/coco128/images/val2017 não encontrado. Certifique-se de que coco128 foi descompactado corretamente.
Filtrando e remapeando rótulos...
Aviso: /content/datasets/coco128/labels/val2017 não encontrado. Certifique-se de que os rótulos de coco128 existem.
Preparação do dataset customizado concluída.


In [8]:
# 9. Criar o arquivo de configuração do dataset para suas classes (CORRIGIDO NOVAMENTE!)
%%writefile data/coco_traffic_signs.yaml
path: ../datasets/traffic_signs_coco128 # Continua apontando para o seu dataset preparado
train: images/train2017
val: images/train2017   # CORRIGIDO: Usar as mesmas imagens de treino para validação

nc: 2
names: [
  'traffic light',
  'stop sign'
]

Writing data/coco_traffic_signs.yaml


In [9]:
# 10. Baixar pesos pré-treinados do YOLOv5s
!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt -P /content/yolov5/

--2025-07-18 14:31:31--  https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/264818686/eab38592-7168-4731-bdff-ad5ede2002be?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-18T15%3A08%3A55Z&rscd=attachment%3B+filename%3Dyolov5s.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-18T14%3A08%3A25Z&ske=2025-07-18T15%3A08%3A55Z&sks=b&skv=2018-11-09&sig=BQFZyywlJslRZVRs%2F0pZNlO6x3skePnv7x%2FEC9px5uc%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1Mjg0OTM5MiwibmJmIjoxNzUyODQ5MDkyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iLmNvcmU

In [10]:
# 11. Iniciar o Treinamento do Modelo
# GARANTA que NÃO HÁ: --cache-images, --disable-wandb, ou --no-save
!python train.py --img 640 --batch 16 --epochs 100 --data 'data/coco_traffic_signs.yaml' --weights yolov5s.pt --name traffic_signs_run --project runs/train

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
2025-07-18 14:31:49.003600: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752849109.298287    1428 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752849109.382053    1428 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=data/coco_traffic_signs.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size